Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Library Imports

In [1]:
data_lake_account_name = ''
file_system_name = 'data'

table_name = "c360_data.preparedinferencedata"

subscription_id = "" 
resource_group = "" 
workspace_name = "" 


StatementMeta(spark1, 52, 1, Finished, Available)

In [4]:
import azureml.core
#print("SDK Version:", azureml.core.VERSION)
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl.run import AutoMLRun
from azureml.train.automl import AutoMLConfig
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

from azureml.core import Workspace


StatementMeta(spark1, 52, 4, Finished, Available)

# Connect to AML Workspace 

In [5]:
#save the model to a local file
import azureml.core

from azureml.core import Workspace
ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)
ws.write_config()   

from azureml.core.model import Model

import joblib
model_name='lease_renewal_model'
model_path = Model.get_model_path(model_name=model_name, _workspace=ws)
loaded_model = joblib.load(model_path)
print('model loaded!')


StatementMeta(spark1, 52, 5, Finished, Available)

model loaded!

In [6]:
sql_str = "SELECT cid ,pid ,uid ,InitialLeaseTerm ,(InitialLeaseTerm + ifnull((num_renewals * avg_renewal_leaseterm),0)) as TotalLeaseTerm,isRenewed, \
WO_AirConditioning ,WO_AirDuctsorVents ,WO_CabinetDoors ,WO_Cabinets ,WO_Carpet , \
WO_Ceiling ,WO_CeilingFan ,WO_CleanUnit ,WO_Cleaning ,WO_Dishwasher ,WO_DoorRepairorReplace ,WO_Doorbell , \
WO_Drain ,WO_Dryer ,WO_Faucet ,WO_FilterReplacement ,WO_GarageRemote ,WO_GarbageDisposal ,WO_Heating ,WO_LightBulbs , \
WO_LightFixture ,WO_LockorExteriorDoor ,WO_LockorInteriorDoor ,WO_Microwave ,WO_Other ,WO_Outlets ,WO_Paint ,WO_PestControl , \
WO_Refrigerator ,WO_ShelvesorDrawers ,WO_ShowerHead ,WO_Sink ,WO_SinkStopper ,WO_SmokeDetector ,WO_StoveorOven ,WO_Switches , \
WO_Thermostat ,WO_Toilet ,WO_TowelBar ,WO_TrashRemoval ,WO_TuborShower ,WO_UnderSink ,WO_Walls ,WO_Washer ,WO_Windows , \
Movein_Easeofprocess ,Movein_OverallSatisfaction ,Movein_StaffCommunication ,Movein_StaffCourtesy ,Movein_StaffProfessionalism , \
Renewal_ConditionofProperty,Renewal_MaintenanceOverallSatisfaction ,Renewal_ManagementCommunication ,Renewal_ManagementCourtesy , \
Renewal_ManagementProblemResolution ,Renewal_ManagementProfessionalism ,Renewal_OverallSatisfaction FROM " + table_name +  \
" where isMovedOut = 'N'"

df_test = spark.sql(sql_str)
df_test_pd = df_test.toPandas()
df_test_pd.shape

StatementMeta(spark1, 52, 6, Finished, Available)

(319, 63)

In [7]:
#batch inferencing
drop_cols = ['cid','pid','uid','isRenewed']
df_test = df_test_pd.drop(drop_cols, axis=1)

model_path = Model.get_model_path(model_name = 'lease_renewal_model')
model = joblib.load(model_path)

prediction = model.predict(df_test)
predit_proba = model.predict_proba(df_test)

df_result = df_test_pd[['cid','pid','uid']] 
df_result['prediction'] = prediction
df_result['predict_proba'] = predit_proba[['Y']]

df_sp = spark.createDataFrame(df_result)
df_sp.write.option('header', 'true').mode('overwrite').csv(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/resident360/batchpredictions/')

df_sp.write.mode("overwrite").saveAsTable("c360_data.lease_renewal_predictions_temp")

StatementMeta(spark1, 52, 7, Finished, Available)

In [11]:
sql_str = '''select CustomerId, min(prediction) as RenewalPrediction, round(min(predict_proba),4) * 100 as RenewalPredictionScore from c360_data.lease_renewal_predictions_temp as pred
inner join c360_data.customer_profile_ids_combined as ids on pred.cid = ids.cid
group by CustomerId'''

df_pred = spark.sql(sql_str)
df_pred.write.mode("overwrite").saveAsTable("c360_data.lease_renewal_predictions")
df_pred.write.mode("overwrite").saveAsTable("c360_data.lease_renewal_prediction")
spark.sql('drop table c360_data.lease_renewal_predictions_temp')

StatementMeta(spark1, 52, 11, Finished, Available)

DataFrame[]